In [1]:
import gensim
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\01din\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

This was also more of an experiment where I wanted to get more familiar with w2v and the dataset. Nothing very useful was made here.

In [63]:
from ast import literal_eval
generic = lambda x: literal_eval(x)
conv = {'nutrition' : generic, 'steps' : generic, 'ingredients' : generic, 'tags' : generic}
df = pd.read_csv('C:/Users/01din\Documents/University\BSc thesis\data\RAW_recipes.csv\RAW\RAW_recipes.csv', converters=conv)

In [86]:
import string

def remove_punctuation(text):
    return ' '.join(char if char not in string.punctuation else ' ' for char in text)

def preprocess_ingredients(ingredients_list):
    preprocessed = []
    for ingredient in ingredients_list:
        words = ingredient.split()
        if len(words) > 1:
            preprocessed.append('_'.join(words))
        else:
            preprocessed.append(ingredient)
    return preprocessed

def replace_multiword_ingredients(instruction, ingredient_list):
    for ingredient in ingredient_list:
        words = ingredient.split()
        if len(words) > 1:
            underscored_ingredient = '_'.join(words)
            instruction = instruction.replace(ingredient, underscored_ingredient)
    return instruction

ingredients = df.ingredients.tolist()
instructions = df.steps.tolist()

instructions = [remove_punctuation(instruction) for instruction in instructions]

recipes = {'ingredients': ingredients, 'instructions': instructions}

preprocessed_ingredients = [preprocess_ingredients(ingredient_list) for ingredient_list in recipes['ingredients']]

preprocessed_instructions = []
for instruction, ingredient_list in zip(instructions, preprocessed_ingredients):
    preprocessed_instructions.append(replace_multiword_ingredients(instruction, ingredient_list))

full_text = [ingredients + [instructions] for ingredients, instructions in zip(preprocessed_ingredients, preprocessed_instructions)]
full_text = [' '.join(sublist) for sublist in full_text]

tokenized_text = [recipe.split() for recipe in full_text]
tokenized_text = [[s for s in recipe if s != ','] for recipe in tokenized_text]

In [87]:
tokenized_text[0]

['winter_squash',
 'mexican_seasoning',
 'mixed_spice',
 'honey',
 'butter',
 'olive_oil',
 'salt',
 'make',
 'a',
 'choice',
 'and',
 'proceed',
 'with',
 'recipe',
 'depending',
 'on',
 'size',
 'of',
 'squash',
 'cut',
 'into',
 'half',
 'or',
 'fourths',
 'remove',
 'seeds',
 'for',
 'spicy',
 'squash',
 'drizzle',
 'olive',
 'oil',
 'or',
 'melted',
 'butter',
 'over',
 'each',
 'cut',
 'squash',
 'piece',
 'season',
 'with',
 'mexican',
 'seasoning',
 'mix',
 'ii',
 'for',
 'sweet',
 'squash',
 'drizzle',
 'melted',
 'honey',
 'butter',
 'grated',
 'piloncillo',
 'over',
 'each',
 'cut',
 'squash',
 'piece',
 'season',
 'with',
 'sweet',
 'mexican',
 'spice',
 'mix',
 'bake',
 'at',
 '350',
 'degrees',
 'again',
 'depending',
 'on',
 'size',
 'for',
 '40',
 'minutes',
 'up',
 'to',
 'an',
 'hour',
 'until',
 'a',
 'fork',
 'can',
 'easily',
 'pierce',
 'the',
 'skin',
 'be',
 'careful',
 'not',
 'to',
 'burn',
 'the',
 'squash',
 'especially',
 'if',
 'you',
 'opt',
 'to',
 'use'

In [88]:
import logging

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

In [89]:
from gensim.models import Word2Vec
new_w2v_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, epochs=50)

2023-04-14 04:33:07,306 : INFO : collecting all words and their counts
2023-04-14 04:33:07,308 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-04-14 04:33:07,453 : INFO : PROGRESS: at sentence #10000, processed 1046736 words, keeping 17892 word types
2023-04-14 04:33:07,561 : INFO : PROGRESS: at sentence #20000, processed 2082452 words, keeping 24744 word types
2023-04-14 04:33:07,667 : INFO : PROGRESS: at sentence #30000, processed 3152956 words, keeping 30079 word types
2023-04-14 04:33:07,773 : INFO : PROGRESS: at sentence #40000, processed 4213730 words, keeping 34763 word types
2023-04-14 04:33:07,886 : INFO : PROGRESS: at sentence #50000, processed 5353583 words, keeping 38513 word types
2023-04-14 04:33:07,993 : INFO : PROGRESS: at sentence #60000, processed 6429441 words, keeping 42381 word types
2023-04-14 04:33:08,096 : INFO : PROGRESS: at sentence #70000, processed 7400876 words, keeping 45240 word types
2023-04-14 04:33:08,195 : INFO : PROGRE

In [4]:
def find_similar_ingredient(ingredient, model, topn=10):
    most_similar = model.wv.most_similar(ingredient, topn=topn)
    return most_similar if most_similar else None

In [8]:
find_similar_ingredient('sausage', new_w2v_model)

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [90]:
new_w2v_model.save("C:/Users/01din\Documents/University\BSc thesis\data/nlp\w2v/custom_word2vec_model.model")

2023-04-14 04:37:43,274 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'C:/Users/01din\\Documents/University\\BSc thesis\\data/nlp\\w2v/custom_word2vec_model.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-14T04:37:43.274624', 'gensim': '4.3.0', 'python': '3.8.8 (tags/v3.8.8:024d805, Feb 19 2021, 13:18:16) [MSC v.1928 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-14 04:37:43,275 : INFO : not storing attribute cum_table
2023-04-14 04:37:43,368 : INFO : saved C:/Users/01din\Documents/University\BSc thesis\data/nlp\w2v/custom_word2vec_model.model


In [2]:
model = gensim.models.Word2Vec.load("C:/Users/01din\Documents/University\BSc thesis\data/nlp\w2v/custom_word2vec_model.model")

In [5]:
find_similar_ingredient('spinach', model)

[('kale', 0.7149818539619446),
 ('broccoli', 0.6888524293899536),
 ('chard', 0.641985297203064),
 ('arugula', 0.6373889446258545),
 ('orzo', 0.6292614340782166),
 ('greens', 0.6251826882362366),
 ('artichokes', 0.6160466074943542),
 ('tortellini', 0.6136913299560547),
 ('escarole', 0.6093213558197021),
 ('asparagus', 0.6003960967063904)]